In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if not torch.cuda.is_available():
    print("Warning CUDA not Found. Using CPU")

# Hyper-parameters
num_epochs = 8
learning_rate = 1e-3

In [ ]:
from google.colab import files
import zipfile
import os
uploaded = files.upload()

zip_path = list(uploaded.keys())[0]
extract_dir = "/content/data"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("extracted to", extract_dir)

In [ ]:
import os, glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class PngSlices(Dataset):
    def __init__(self, folder, size=(28,28)):
        self.fs = sorted(glob.glob(os.path.join(folder, "*.png")))
        if not self.fs:
            raise FileNotFoundError(f"No PNGs found in {folder}")
        self.tf = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize(size, antialias=True),
            transforms.ToTensor(),
        ])

    def __len__(self): return len(self.fs)

    def __getitem__(self, i):
        img = Image.open(self.fs[i]).convert("L")
        return self.tf(img)

trainset = PngSlices("/content/data/keras_png_slices_data/keras_png_slices_train", size=(28,28))
testset  = PngSlices("/content/data/keras_png_slices_data/keras_png_slices_test",  size=(28,28))

train_loader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
test_loader  = DataLoader(testset,  batch_size=100, shuffle=False, num_workers=2)

print(f"> OASIS ready: {len(trainset)} train slices, {len(testset)} test slices")
